# 🧠 Tutorial: BCI Competition IV Dataset 4 Exploration
## Working with ECoG Brain Signal Data

---

**Learning Objectives:**
- Download and load the BCI Competition IV Dataset 4 from braindecode
- Explore ECoG (Electrocorticography) signal structure
- Understand channel configurations and sampling rates
- Visualize neural signals in time and frequency domains
- Analyze signal characteristics across different brain channels

---


## 📥 Step 1: Data Download

We'll download the BCI Competition IV Dataset 4, which contains ECoG recordings from patients performing finger movements.

**Data Source:** [BCI Competition IV](http://www.bbci.de/competition/iv/) - Dataset 4
- **Dataset:** ECoG recordings from 3 patients
- **Task:** Finger flexion movements (5 fingers)
- **Channels:** Multiple ECoG electrodes recording brain activity


In [ ]:
# Install required packages if not already installed
# Uncomment the lines below if you need to install packages
# !uv add braindecode moabb
# !uv pip install -r requirements.txt


In [ ]:
# Download and load BCI Competition IV Dataset 4
import os
from pathlib import Path
from braindecode.datasets import BCICompetitionIVDataset4

# Hardcoded dataset path
MNE_DATA_PATH = Path("/workspace/RandomAssembly/mne_data")

# Create dataset directory if it doesn't exist
MNE_DATA_PATH.mkdir(parents=True, exist_ok=True)

# Set MNE_DATA_PATH environment variable
# This tells MOABB/braindecode where to download the data
os.environ['MNE_DATA_PATH'] = str(MNE_DATA_PATH)


print(f"✓ Set MNE_DATA_PATH to: {os.environ['MNE_DATA_PATH']}")
print(f"✓ Directory exists: {MNE_DATA_PATH.exists()}")

# Download dataset if not already available
print("\nDownloading BCI Competition IV Dataset 4...")
try:
    BCICompetitionIVDataset4.download()
    print("✓ Dataset download complete!")
except Exception as e:
    print(f"⚠ Download error: {e}")
    print("Dataset may already be downloaded or there was a connection issue.")

# Check dataset location
# MOABB stores datasets in moabb/BCICIV4 subdirectory
dataset_path = MNE_DATA_PATH / "moabb" / "BCICIV4"
print(f"\nDataset storage information:")
print(f"  - MNE_DATA_PATH: {MNE_DATA_PATH}")
print(f"  - Dataset path: {dataset_path}")
print(f"  - Directory exists: {dataset_path.exists()}")

if dataset_path.exists():
    contents = list(dataset_path.iterdir())
    if contents:
        print(f"  - Found {len(contents)} items in dataset directory")
    else:
        print(f"  - Directory exists but is empty")
else:
    print(f"  - Dataset will be downloaded to: {dataset_path}")


/opt/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Dataset download complete!


ModuleNotFoundError: No module named 'main'

## 📊 Step 2: Load and Inspect Dataset Structure

Now we'll load the dataset and explore its structure, including the number of subjects, channels, and recording characteristics.


In [ ]:
# Load dataset for subject 1 (you can change this to [1, 2, 3] for all subjects)
subject_ids = 1  # Can be 1, 2, 3, or [1, 2, 3] for all subjects

print(f"Loading dataset for subject(s): {subject_ids}")
dataset = BCICompetitionIVDataset4(subject_ids=subject_ids)

print(f"\n✓ Dataset loaded successfully!")
print(f"  - Number of recordings: {len(dataset.datasets)}")
print(f"  - Dataset type: {type(dataset).__name__}")


### 🔍 Quick Data Inspection


In [ ]:
# Explore the first recording
if len(dataset.datasets) > 0:
    first_recording = dataset.datasets[0]
    print(f"First recording type: {type(first_recording).__name__}")
    print(f"First recording description:\n{first_recording.description}")
    
    # Get raw data
    raw = first_recording.raw
    print(f"\n📊 Raw Data Information:")
    print(f"  - Number of channels: {len(raw.ch_names)}")
    print(f"  - Sampling frequency: {raw.info['sfreq']} Hz")
    print(f"  - Duration: {raw.times[-1]:.2f} seconds")
    print(f"  - Number of time points: {len(raw.times)}")
    print(f"  - Channel names (first 10): {raw.ch_names[:10]}")
else:
    print("⚠ No recordings found in dataset")


In [ ]:
# First, check what channel types are available
print("Available channel types in raw data:")
print(f"  - Channel names: {raw.ch_names[:10]}..." if len(raw.ch_names) > 10 else f"  - Channel names: {raw.ch_names}")

# Check channel types
channel_types = [raw.get_channel_types()[i] for i in range(len(raw.ch_names))]
unique_types = set(channel_types)
print(f"  - Unique channel types: {unique_types}")

# Count each type
for ch_type in unique_types:
    count = channel_types.count(ch_type)
    print(f"    * {ch_type}: {count} channels")

# Extract ECoG data
try:
    ecog_picks = raw.pick_types(ecog=True)
    ecog_data, ecog_times = ecog_picks[:, :]
    ecog_ch_names = ecog_picks.ch_names
    print(f"\n✓ ECoG data extracted: {ecog_data.shape[0]} channels, {ecog_data.shape[1]} time points")
except Exception as e:
    print(f"\n⚠ Error extracting ECoG: {e}")
    # Fallback: use all channels if ECoG picking fails
    ecog_data, ecog_times = raw[:, :]
    ecog_ch_names = raw.ch_names
    print(f"  Using all channels as ECoG: {ecog_data.shape[0]} channels")

# Extract target data (finger flexions)
# According to braindecode example, targets are stored as 'misc' channels
# But they might not be available without preprocessing
target_data = None
target_times = None
target_ch_names = None

try:
    target_picks = raw.pick_types(misc=True)
    target_data, target_times = target_picks[:, :]
    target_ch_names = target_picks.ch_names
    print(f"✓ Target data extracted: {target_data.shape[0]} channels, {target_data.shape[1]} time points")
except ValueError as e:
    print(f"\n⚠ No 'misc' channels found for targets: {e}")
    print("  Note: Target channels may need preprocessing or may be stored differently.")
    print("  For now, we'll work with ECoG data only.")
    print("  You may need to preprocess the data first (see braindecode example).")

print(f"\n📈 ECoG Data Shape: {ecog_data.shape}")
print(f"  - ECoG Channels: {ecog_data.shape[0]}")
print(f"  - Time points: {ecog_data.shape[1]}")
print(f"  - Time range: {ecog_times[0]:.2f} to {ecog_times[-1]:.2f} seconds")

if target_data is not None:
    print(f"\n📊 Target Data Shape: {target_data.shape}")
    print(f"  - Target channels (fingers): {target_data.shape[0]}")
    print(f"  - Target channel names: {target_ch_names}")
    print(f"  - Target sampling frequency: {raw.info.get('temp', {}).get('target_sfreq', 'N/A')} Hz")
else:
    print(f"\n📊 Target Data: Not available (no misc channels found)")

print(f"\n📊 ECoG Data Statistics:")
print(f"  - Mean: {ecog_data.mean():.4f}")
print(f"  - Std: {ecog_data.std():.4f}")
print(f"  - Min: {ecog_data.min():.4f}")
print(f"  - Max: {ecog_data.max():.4f}")

if target_data is not None:
    print(f"\n📊 Target Data Statistics:")
    print(f"  - Mean: {target_data.mean():.4f}")
    print(f"  - Std: {target_data.std():.4f}")
    print(f"  - Min: {target_data.min():.4f}")
    print(f"  - Max: {target_data.max():.4f}")


## 📏 Step 3: Store Data in Pandas DataFrames

Let's organize the ECoG signals and finger flexion targets into pandas DataFrames for easier analysis.


In [ ]:
import numpy as np
import pandas as pd

# Create DataFrame for ECoG time series data
# Each column is a channel, each row is a time point
df_ecog = pd.DataFrame(ecog_data.T, columns=ecog_ch_names, index=ecog_times)
df_ecog.index.name = 'time_seconds'

print("✓ ECoG DataFrame created")
print(f"  - Shape: {df_ecog.shape}")
print(f"  - Columns (first 5): {list(df_ecog.columns[:5])}")
print(f"\nFirst few rows:")
print(df_ecog.head())

# Create DataFrame for target (finger flexion) time series if available
df_targets = None
finger_names = None

if target_data is not None and target_ch_names is not None:
    finger_names = target_ch_names
    df_targets = pd.DataFrame(target_data.T, columns=finger_names, index=target_times)
    df_targets.index.name = 'time_seconds'
    
    print(f"\n✓ Target DataFrame created")
    print(f"  - Shape: {df_targets.shape}")
    print(f"  - Columns (fingers): {list(df_targets.columns)}")
    print(f"\nFirst few rows:")
    print(df_targets.head())
else:
    print(f"\n⚠ Target DataFrame not created - no target data available")
    print("  You may need to preprocess the data first to access target channels.")
    print("  See the braindecode example for preprocessing steps.")


In [ ]:
# Create summary DataFrame for channel statistics
channel_info = []
for i, ch_name in enumerate(ecog_ch_names):
    channel_data = ecog_data[i, :]
    channel_info.append({
        'channel_index': i,
        'channel_name': ch_name,
        'channel_type': 'ECoG',
        'mean': channel_data.mean(),
        'std': channel_data.std(),
        'min': channel_data.min(),
        'max': channel_data.max(),
        'range': channel_data.max() - channel_data.min()
    })

# Add target channel info
for i, ch_name in enumerate(finger_names):
    target_channel_data = target_data[i, :]
    channel_info.append({
        'channel_index': i + len(ecog_ch_names),
        'channel_name': ch_name,
        'channel_type': 'Target',
        'mean': target_channel_data.mean(),
        'std': target_channel_data.std(),
        'min': target_channel_data.min(),
        'max': target_channel_data.max(),
        'range': target_channel_data.max() - target_channel_data.min()
    })

df_channels = pd.DataFrame(channel_info)

print("✓ Channel information DataFrame created")
print(f"\nChannel Statistics Summary:")
print(df_channels.groupby('channel_type').describe())


## 📊 Step 4: Data Visualization

Let's create comprehensive visualizations to understand the ECoG signals in both time and frequency domains.


### 4.1 Setup Visualization Libraries


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal

# Set style for prettier plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['figure.dpi'] = 100

print("✓ Visualization libraries ready")


### 4.2 Time Series Visualization - Sample Channels


In [ ]:
# Plot first 10 seconds of multiple channels
n_channels_to_plot = min(10, len(raw.ch_names))
time_mask = times <= 10.0  # First 10 seconds

fig, ax = plt.subplots(figsize=(16, 8))

for i in range(n_channels_to_plot):
    # Normalize for visualization
    channel_data = data[i, time_mask]
    channel_data_norm = (channel_data - channel_data.mean()) / (channel_data.std() + 1e-8)
    ax.plot(times[time_mask], channel_data_norm + i * 2, 
            label=raw.ch_names[i], alpha=0.7, linewidth=1)

ax.set_xlabel('Time (seconds)', fontsize=12)
ax.set_ylabel('Channel (normalized amplitude)', fontsize=12)
ax.set_title(f'ECoG Signals - First {n_channels_to_plot} Channels (First 10 seconds)', 
             fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=8, ncol=2)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"✓ Displayed {n_channels_to_plot} channels over first 10 seconds")


### 4.3 Finger Flexion Targets Visualization

Visualize the finger flexion targets (time series) as shown in the braindecode example.


In [ ]:
# Plot finger flexion targets over time (if available)
if df_targets is not None and finger_names is not None:
    # Use first 30 seconds for visualization (as in braindecode example)
    time_limit = 30.0
    time_mask = df_targets.index <= time_limit

    fig, axes = plt.subplots(len(finger_names), 1, figsize=(14, 3*len(finger_names)), sharex=True)

    if len(finger_names) == 1:
        axes = [axes]  # Make it iterable for single finger case

    for i, finger in enumerate(finger_names):
        axes[i].plot(df_targets.index[time_mask], df_targets[finger][time_mask], 
                     linewidth=2, label=f'{finger}', color=plt.cm.tab10(i))
        axes[i].set_ylabel('Finger Flexion', fontsize=11)
        axes[i].set_title(f'{finger}', fontsize=12, fontweight='bold')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)

    axes[-1].set_xlabel('Time (seconds)', fontsize=12)
    fig.suptitle('Finger Flexion Targets (Time Series)', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

    print(f"✓ Displayed finger flexion targets for {len(finger_names)} fingers")
    print(f"  - Time range: 0 to {time_limit} seconds")
    print(f"  - Target sampling frequency: {raw.info.get('temp', {}).get('target_sfreq', 'N/A')} Hz")
else:
    print("⚠ Target data not available for visualization")
    print("  Target channels may need preprocessing first.")
    print("  See the braindecode example for preprocessing steps to access finger flexion targets.")


### 4.4 Power Spectral Density Analysis


In [ ]:
# Compute and plot power spectral density for a sample ECoG channel
sample_channel_idx = 0
sample_channel_data = ecog_data[sample_channel_idx, :]

# Compute power spectral density using Welch's method
freqs, psd = signal.welch(sample_channel_data, fs=raw.info['sfreq'], nperseg=1024)

fig, ax = plt.subplots(figsize=(14, 6))
ax.semilogy(freqs, psd, linewidth=2, color='steelblue')
ax.set_xlabel('Frequency (Hz)', fontsize=12)
ax.set_ylabel('Power Spectral Density', fontsize=12)
ax.set_title(f'Power Spectral Density - Channel: {ecog_ch_names[sample_channel_idx]}', 
             fontsize=14, fontweight='bold')
ax.set_xlim(0, 100)  # Focus on 0-100 Hz range (most relevant for neural signals)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Find peak frequency
peak_freq_idx = np.argmax(psd[(freqs >= 1) & (freqs <= 100)])
peak_freq = freqs[(freqs >= 1) & (freqs <= 100)][peak_freq_idx]
print(f"✓ Peak frequency: {peak_freq:.2f} Hz")


### 4.5 Channel Variability Analysis


In [ ]:
# Plot standard deviation across all ECoG channels
ecog_channels_df = df_channels[df_channels['channel_type'] == 'ECoG']

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar plot of channel standard deviations
axes[0].bar(range(len(ecog_channels_df)), ecog_channels_df['std'], 
            alpha=0.7, color='coral', edgecolor='black')
axes[0].set_xlabel('Channel Index', fontsize=12)
axes[0].set_ylabel('Standard Deviation', fontsize=12)
axes[0].set_title('Signal Variability Across ECoG Channels', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Histogram of channel statistics
axes[1].hist(ecog_channels_df['std'], bins=30, color='seagreen', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Standard Deviation', fontsize=12)
axes[1].set_ylabel('Number of Channels', fontsize=12)
axes[1].set_title('Distribution of ECoG Channel Variability', fontsize=14, fontweight='bold')
axes[1].axvline(ecog_channels_df['std'].median(), color='red', linestyle='--', linewidth=2,
                label=f'Median: {ecog_channels_df["std"].median():.4f}')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"✓ Channel variability analysis complete")
most_var = ecog_channels_df.loc[ecog_channels_df['std'].idxmax()]
least_var = ecog_channels_df.loc[ecog_channels_df['std'].idxmin()]
print(f"  - Most variable channel: {most_var['channel_name']} (std: {most_var['std']:.4f})")
print(f"  - Least variable channel: {least_var['channel_name']} (std: {least_var['std']:.4f})")


### 4.6 Target Distribution Analysis

Analyze the distribution of finger flexion values.


In [ ]:
# Plot distributions of finger flexion targets (if available)
if df_targets is not None and finger_names is not None:
    fig, axes = plt.subplots(1, len(finger_names), figsize=(4*len(finger_names), 5))
    if len(finger_names) == 1:
        axes = [axes]

    for i, finger in enumerate(finger_names):
        axes[i].hist(df_targets[finger], bins=50, alpha=0.7, color=plt.cm.tab10(i), edgecolor='black')
        axes[i].set_title(f'{finger}', fontsize=12, fontweight='bold')
        axes[i].set_xlabel('Finger Flexion Value', fontsize=11)
        axes[i].set_ylabel('Frequency', fontsize=11)
        axes[i].axvline(df_targets[finger].mean(), color='red', linestyle='--', linewidth=2,
                        label=f'Mean: {df_targets[finger].mean():.4f}')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3, axis='y')

    plt.suptitle('Distribution of Finger Flexion Targets', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

    print("✓ Target distribution analysis complete")
    for finger in finger_names:
        print(f"  - {finger}: mean={df_targets[finger].mean():.4f}, std={df_targets[finger].std():.4f}")
else:
    print("⚠ Target data not available for distribution analysis")


### 4.7 Correlation Between ECoG Channels


In [ ]:
# Compute correlation matrix for a subset of ECoG channels (for performance)
n_channels_corr = min(20, len(ecog_ch_names))

# Sample every 100th time point for faster computation
sample_indices = np.arange(0, df_ecog.shape[0], 100)
corr_data = df_ecog.iloc[sample_indices, :n_channels_corr]

# Compute correlation
correlation_matrix = corr_data.corr().values

# Plot correlation heatmap
fig, ax = plt.subplots(figsize=(12, 10))
im = ax.imshow(correlation_matrix, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
ax.set_xticks(range(n_channels_corr))
ax.set_yticks(range(n_channels_corr))
ax.set_xticklabels(ecog_ch_names[:n_channels_corr], rotation=45, ha='right', fontsize=8)
ax.set_yticklabels(ecog_ch_names[:n_channels_corr], fontsize=8)
ax.set_title(f'ECoG Channel Correlation Matrix (First {n_channels_corr} channels)', 
             fontsize=14, fontweight='bold')
plt.colorbar(im, ax=ax, label='Correlation Coefficient')
plt.tight_layout()
plt.show()

print(f"✓ Correlation analysis complete for {n_channels_corr} ECoG channels")


### 4.8 Correlation Between Targets (Fingers)

Analyze how finger flexions correlate with each other.


In [ ]:
# Compute correlation between finger flexions (if available)
if df_targets is not None and finger_names is not None and len(finger_names) > 1:
    target_correlation = df_targets.corr()

    # Plot correlation heatmap
    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(target_correlation.values, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
    ax.set_xticks(range(len(finger_names)))
    ax.set_yticks(range(len(finger_names)))
    ax.set_xticklabels(finger_names, rotation=45, ha='right')
    ax.set_yticklabels(finger_names)
    ax.set_title('Correlation Between Finger Flexions', fontsize=14, fontweight='bold')
    plt.colorbar(im, ax=ax, label='Correlation Coefficient')

    # Add correlation values as text
    for i in range(len(finger_names)):
        for j in range(len(finger_names)):
            text = ax.text(j, i, f'{target_correlation.iloc[i, j]:.2f}',
                          ha="center", va="center", color="black", fontsize=10)

    plt.tight_layout()
    plt.show()

    print("✓ Target correlation analysis complete")
    print(f"\nCorrelation matrix:")
    print(target_correlation)
else:
    print("⚠ Target data not available or insufficient for correlation analysis")
    if df_targets is not None and len(finger_names) == 1:
        print("  Only one finger target available - correlation not applicable")


## 💾 Step 5: Save Processed Data

Let's save the processed channel information for future use.


In [ ]:
# Save channel information to CSV
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

output_file = output_dir / "bci_channel_info.csv"
df_channels.to_csv(output_file, index=False)

print(f"✓ Channel information saved to: {output_file}")
print(f"  - Total channels: {len(df_channels)}")
print(f"  - Columns: {list(df_channels.columns)}")

# Also save a summary
summary = {
    'subject_id': subject_ids,
    'num_channels': len(raw.ch_names),
    'sampling_freq': raw.info['sfreq'],
    'duration_seconds': times[-1],
    'num_timepoints': len(times),
    'data_mean': data.mean(),
    'data_std': data.std(),
    'data_min': data.min(),
    'data_max': data.max()
}

summary_df = pd.DataFrame([summary])
summary_file = output_dir / "bci_dataset_summary.csv"
summary_df.to_csv(summary_file, index=False)

print(f"✓ Dataset summary saved to: {summary_file}")


---

## 🎯 Ready for Time Series Forecasting!

Your ECoG data is now explored and ready for forecasting:

**Options:**
- Use individual channels for univariate time series forecasting
- Aggregate multiple channels (mean/median) for composite signals
- Apply TimesFM or other forecasting models
- Use alternative methods (Linear Regression) on Apple Silicon

**Next steps:**
- Run `main.py` for complete forecasting pipeline
- Explore different channel combinations
- Analyze frequency domain features
- Build predictive models for finger movement decoding

---


---

## 🎉 Exploration Complete!

**What we accomplished:**
1. ✅ Downloaded and loaded BCI Competition IV Dataset 4
2. ✅ Explored dataset structure and channel information
3. ✅ Visualized ECoG signals in time domain
4. ✅ Analyzed power spectral density
5. ✅ Examined channel variability and correlations
6. ✅ Saved processed data for future use

**Key Findings:**
- Dataset contains multi-channel ECoG recordings
- Signals sampled at high frequency (1000 Hz typical)
- Channels show varying levels of activity
- Ready for time series forecasting analysis

---
